## Trends.Earth productivity Notebook v2 <a name="top"></a>

Conversion of [Trends.Earth productivity](http://trends.earth/docs/en/background/understanding_indicators.html#productivity) [script](https://github.com/ConservationInternational/landdegradation/blob/master/landdegradation/productivity.py) into a Jupyter Notebook.

<img src="http://trends.earth/docs/en/_images/indicator_15_3_1_prod_subindicators.png" alt="" width="600"/>   
   
    
- [Load mean annual NDVI](#load_data) (and quality dataset) from saved netcdf file - section

  generated by [BC_NDVI_annual_mean.ipynb](BC_NDVI_annual_mean.ipynb) and stored in work_space.
    

- [Visualize mean annual NDVI through time (OPTIONAL)](#visualize_data) section (and quality dataset) - section


- [Initiate output dataset](#init_ds) section


- [Compute trajectory](#trajectory) - section
    
<a href="http://trends.earth/docs/en/background/understanding_indicators.html#productivity-trajectory"><img src="http://trends.earth/docs/en/_images/lp_traj_flow.PNG" alt="" width="800"/></a>

- [Compute performance](#performance) - section

<a href="http://trends.earth/docs/en/background/understanding_indicators.html#productivity-performance"><img src="http://trends.earth/docs/en/_images/lp_perf_flow.PNG" alt="" width="800"/></a>

- [Compute state](#state) - section

<a href="http://trends.earth/docs/en/background/understanding_indicators.html#productivity-state"><img src="http://trends.earth/docs/en/_images/lp_state_flow.PNG" alt="" width="800"/></a>

- [Combinine productivity indicators](#combine) - section

<a href="http://trends.earth/docs/en/background/understanding_indicators.html#combining-productivity-indicators"><img src="http://trends.earth/docs/en/_images/lp_aggregation.PNG" alt="" width="800"/></a>

- [Visualize productivity (OPTIONAL)](#visualize) - section

**Requirements:**
- ./swiss_utils/data_cube_utilities/sdc_utilities.py
- Land Cover: ./auxiliary_data/CH_lc_TE.tif (SWISS only)
- Soil Taxonomy: ./auxiliary_data/CH_soil_tax_TE.tif (Swiss only)
- NDVI annual mean netcdf (stored in work_path) from ITI_NDVI_annual_mean_v2.ipynb

**Changelog:**
- better management of nodata
- TAXNWRB_250m_ll.tif and ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992_2015-v2.0.7.tif respectively replaced by CH_soil_tax_TE.tif and CH_lc_TE.tif (back-processed from Trends.Earth plugin output for all Switzerland) as it appears the plugin is not using files mentionned in documentation.
- minor changes

---


In [ ]:
# Import dependencies

%matplotlib inline

import os
import sys
import calendar
import uuid
import rasterio
import gdal
import osr

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import colors
from pandas import DataFrame

from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    ImageOverlay,
    DrawControl,
    LayersControl
)

import datacube
dc = datacube.Datacube()

from utils.data_cube_utilities.dc_display_map import display_map, _degree_to_zoom_level
from utils.data_cube_utilities.dc_utilities import write_single_band_png_from_xr

from swiss_utils.data_cube_utilities.sdc_utilities import printandlog, load_multi_clean

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Configuration section

work_path = '/datacube/ui_results/notebook/BC_switzerland_0112_LS58'

start_year = 2001
bl_end_year = 2012 # Baseline end
tg_start_year = 2013 # Target start
end_year = 2015

# Performance calculation data
lc_path = './auxiliary_data/CH_lc_TE.tif'
soil_path = './auxiliary_data/CH_soil_tax_TE.tif'

log_name = 'BC_TendsEarth_productivity_v2.log'
user_mail = 'bruno.chatenoux@unepgrid.ch'


In [ ]:
# Function to export to geotiff

EXT2DRIV_CONVERSION = {
    "tif": "GTiff",
    "nc": "netCDF",
    "img": "HFA"
}

NP2GDAL_CONVERSION = {
      "uint8": 1,
      "int8": 1,
      "uint16": 2,
      "int16": 3,
      "uint32": 4,
      "int32": 5,
      "float32": 6,
      "uint64": 7,
      "int64": 7,
      "float64": 7,
      "complex64": 10,
      "complex128": 11,
    }

def da2raster(da, rast_name, nodata_val = None, md_rast = None, md_band = None, opt = []):
    """
    Description:
      Convert an xarray.DataArray into a proper Geotiff (no shift, CRS (epsg:4326), metadata and nodata value)
    -----
    Input:
      da: xarray.DataArray
      rast_name: string
          Output file name (could be .tif. img or .nc, but the last one will not manage the metadata)
      nodata: number (optional)
      md_rast: dictionnary (optional)
          raster metadata e.g. {'long_name':'productivity indicators', 'configuartion':'Landsat 5,7 and 8 2001-2015'}
      md_band: dictionnary (optional)
          band metadata e.g. {'long_name':'linear trend', 'units':'NDVI/year'}
      opt: list
          gdal option (depends on used raster format) e.g. ['COMPRESS=DEFLATE']
          see https://www.gdal.org/formats_list.html for options per format
    Output:
      raster file
    """
    
    # identify GDAL Driver using rast_name extent
    driv = EXT2DRIV_CONVERSION[rast_name.split('.')[-1]]
    
    # identify da GDAL data type
    # source: https://borealperspectives.wordpress.com/2014/01/16/data-type-mapping-when-using-pythongdal-to-write-numpy-arrays-to-geotiff/
    gdaltype = NP2GDAL_CONVERSION[da.dtype.name]
    
    # set band name
    band_name = da.name if da.name is not None else 'noname'
    
    # compute georeferencing parameters
    cols = da.shape[1]
    rows = da.shape[0]
    rasterOrigin = (da.longitude.values.min(), da.latitude.values.max()) # Top-Left
    pixelWidth = (da.longitude.values.max() - da.longitude.values.min()) / (cols - 1)
    pixelHeight = (da.latitude.values.max() - da.latitude.values.min()) / (rows - 1)
    origin_left = rasterOrigin[0] - pixelWidth / 2
    origin_top = rasterOrigin[1] + pixelHeight / 2
    
    # Create the geotiff
    driver = gdal.GetDriverByName(driv)
    outRaster = driver.Create(rast_name, cols, rows, 1, gdaltype, options=opt)
    if md_rast:
        outRaster.SetMetadata(md_rast)
    outRaster.SetGeoTransform((origin_left, pixelWidth, 0, origin_top, 0, -pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(da.values)
    outband.SetDescription(band_name)
    if md_band:
        outband.SetMetadata(md_band)
    if nodata_val:
        outband.SetNoDataValue(nodata_val)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()
    
    return 0

### Load mean annual NDVI and quality dataset <a name="load_data"></a>[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

In [ ]:
%%time

# list files in work_path (will not work with subfolders)
for dirpath, dirnames, filenames in os.walk(work_path):
    break

# Compare requested and available years
years = []
noyears = []
for year in range(start_year, end_year + 1):
    if 'NDVI_%s.nc' % (year) in filenames:
        years.append(year)
    else:
        print('Year %s is not available' % (year))
        noyears.append(year)
len_years = len(years)
        
# Load and merge .nc files
NDVI = [None]*len_years
for year_ind, year in enumerate(years):
    print(year)
    da = xr.open_dataset('%s/NDVI_%s.nc' % (work_path, year))
    NDVI[year_ind] = da
    
NDVI = xr.concat(NDVI, dim='time')
NDVI.coords['time'] = years

# Add missing years with nodata
for year in noyears:
    nan_NDVI = NDVI.isel(time=0).where(not NDVI.isel(time=0), 0)
    nan_NDVI['mean'] = nan_NDVI['mean'].where(not nan_NDVI)
    nan_NDVI.coords['time'] = year
    nan_NDVI = nan_NDVI.expand_dims('time')
    NDVI = xr.merge([NDVI, nan_NDVI])

In [ ]:
NDVI

### Visualize mean annual NDVI and quality dataset through time (OPTIONAL) <a name="visualize_data"></a>[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

**_!!! Full section to be commented in case of large dataset !!!_**

### Initiate output dataset <a name="init_ds"></a>[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

In [ ]:
%%time

# quality metadata for geotiff export
md_traj = {'description':'productivity-quality indicators using Landsat 5,7 and 8 over the period 2001-2015'}

# Compute statistic over le full period
ndvi_mean = NDVI['mean']
ndvi_avg = ndvi_mean.mean(dim=['time'])
ndvi_avg.name = 'ndvi_avg'
productivity = ndvi_avg.to_dataset(name = 'ndvi_avg').astype(np.float32)
da2raster(ndvi_avg, '%s/qual_ndvi_avg.tif' % (work_path),
          md_rast = md_traj,
          md_band = {'long_name':'NDVI mean', 'units':'NDVI'},
          opt = ['COMPRESS=DEFLATE'])

ndvi_count = NDVI['count'].sum(dim=['time'])
ndvi_count.name = 'ndvi_count'
productivity = productivity.merge(ndvi_count.to_dataset(name = 'ndvi_count').astype(np.uint16))
da2raster(ndvi_count, '%s/qual_ndvi_count.tif' % (work_path),
          md_rast = md_traj,
          md_band = {'long_name':'data count', 'units':'none'},
          opt = ['COMPRESS=DEFLATE'])

ndvi_pc = np.rint(ndvi_count / np.rint(NDVI['count'] / (NDVI['qual'] / 100)).sum(dim=['time']) * 100).astype(np.uint8)
ndvi_pc.name = 'ndvi_pc'
productivity = productivity.merge(ndvi_pc.to_dataset(name = 'ndvi_pc'))
da2raster(ndvi_pc, '%s/qual_ndvi_pc.tif' % (work_path),
          md_rast = md_traj,
          md_band = {'long_name':'data percentage', 'units':'percentage'},
          opt = ['COMPRESS=DEFLATE'])
del ndvi_count
del ndvi_pc

print(productivity)

### Compute trajectory <a name="trajectory"></a>[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

In [ ]:
%%time
# trajectory metadata for geotiff export
md_traj = {'description':'productivity-trajectory indicators using Landsat 5,7 and 8 over the period 2001-2015'}  

# Calculate mk_trend
def mann_kendall(x, alpha = 0.05):
    n = len(x)

    # calculate S 
    s = 0
    for k in range(n-1):
        for j in range(k+1,n):    
            v = np.sign(x[j] - x[k])
            v = v.fillna(0) # replace nan with 0
            s += v
    return s.astype(np.int16)

mk_trend = mann_kendall(NDVI['mean']).compute()
mk_trend.name = 'mk_trend'
productivity = productivity.merge(mk_trend.to_dataset(name = 'mk_trend'))
da2raster(mk_trend, '%s/traj_mk_trend.tif' % (work_path),
          md_rast = md_traj,
          md_band = {'long_name':'Mann-Kendal', 'units':'none'},
          opt = ['COMPRESS=DEFLATE'])

def da_linreg_params(y, dim = 'time'):
    x = y.where(np.isnan(y), y[dim]) # attribute time to pixel with values

    mean_x = x.mean(dim=dim)
    mean_y = y.mean(dim=dim)
    mean_xx = (x * x).mean(dim=dim)
    mean_xy = (x * y).mean(dim=dim)

    s = ((mean_x * mean_y) - mean_xy) / ((mean_x * mean_x) - mean_xx)
    
    i = mean_y - mean_x * s
    
    return s, i

lin_trend, intercept = da_linreg_params(NDVI['mean'])

lin_trend.name = 'lin_trend'
productivity = productivity.merge(lin_trend.to_dataset(name = 'lin_trend'))
da2raster(lin_trend, '%s/traj_lin_trend.tif' % (work_path), nodata_val = 9999,
          md_rast = md_traj,
          md_band = {'long_name':'linear trend', 'units':'NDVI/year'},
          opt = ['COMPRESS=DEFLATE'])

# Calulate Mann-Kendall significance
def get_kendall_coef(n, level=95):
    # The minus 4 is because the indexing below for a sample size of 4
    assert(n >= 4)
    n = n - 4
    coefs = {90: [4, 6, 7, 9, 10, 12, 15, 17, 18, 22, 23, 27, 28, 32, 35, 37, 40, 42,
                  45, 49, 52, 56, 59, 61, 66, 68, 73, 75, 80, 84, 87, 91, 94, 98, 103,
                  107, 110, 114, 119, 123, 128, 132, 135, 141, 144, 150, 153, 159,
                  162, 168, 173, 177, 182, 186, 191, 197, 202],
             95: [4, 6, 9, 11, 14, 16, 19, 21, 24, 26, 31, 33, 36, 40, 43, 47, 50, 54,
                  59, 63, 66, 70, 75, 79, 84, 88, 93, 97, 102, 106, 111, 115, 120,
                  126, 131, 137, 142, 146, 151, 157, 162, 168, 173, 179, 186, 190,
                  197, 203, 208, 214, 221, 227, 232, 240, 245, 251, 258],
             99: [6, 8, 11, 18, 22, 25, 29, 34, 38, 41, 47, 50, 56, 61, 65, 70, 76, 81,
                  87, 92, 98, 105, 111, 116, 124, 129, 135, 142, 150, 155, 163, 170,
                  176, 183, 191, 198, 206, 213, 221, 228, 236, 245, 253, 260, 268,
                  277, 285, 294, 302, 311, 319, 328, 336, 345, 355, 364]}
    return coefs[level][n]

kendall90 = get_kendall_coef(len_years, 90)
kendall95 = get_kendall_coef(len_years, 95)
kendall99 = get_kendall_coef(len_years, 99)

# np.logical_and used as instead of np.all as is 30 to 40% faster
signif = np.full((mk_trend.shape[0], mk_trend.shape[1]), 9999)
signif = np.where(np.logical_and(lin_trend.values < 0, abs(mk_trend.values) >= kendall90), -1, signif)
signif = np.where(np.logical_and(lin_trend.values < 0, abs(mk_trend.values) >= kendall95), -2, signif)
signif = np.where(np.logical_and(lin_trend.values < 0, abs(mk_trend.values) >= kendall99), -3, signif)
signif = np.where(np.logical_and(lin_trend.values > 0, abs(mk_trend.values) >= kendall90), 1, signif)
signif = np.where(np.logical_and(lin_trend.values > 0, abs(mk_trend.values) >= kendall95), 2, signif)
signif = np.where(np.logical_and(lin_trend.values > 0, abs(mk_trend.values) >= kendall99), 3, signif)
signif = np.where(abs(mk_trend.values) <= kendall90, 0, signif)
signif = np.where(abs(lin_trend.values) <= 0.001, 0, signif)

# Convert signif to a propper DataArray
signif = xr.DataArray(signif, dims=['latitude', 'longitude']).astype(np.int16)
signif = signif.assign_coords(latitude=mk_trend.latitude, longitude=mk_trend.longitude)
signif.name = 'signif'
productivity = productivity.merge(signif.to_dataset(name = 'signif'))
da2raster(signif, '%s/traj_signif.tif' % (work_path), nodata_val = 9999,
          md_rast = md_traj,
          md_band = {'long_name':'significance', 'units':'category'},
          opt = ['COMPRESS=DEFLATE'])
del mk_trend
del lin_trend
del signif

print(productivity)

### Compute performance <a name="performance"></a>[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

In [ ]:
%%time

# performance metadata for geotiff export
md_traj = {'description':'productivity-performance indicators using Landsat 5,7 and 8 over the period 2001-2015'}

# Resample and load Land Cover and Soil taxonomy
def clip_resample_load(xd, tif_path):
    """
    clip, resample and load a geotiff (tif_path) to overlay a given xarray.Dataset or DataArray (xd)
    in EPSG 4326 CRS (reproject if needed).

    Parameters
    ----------
    xd: xarray.Dataset or DataArray
    tif_path: str
        The string filepath to a GeoTIFF.
    """
    
    # generate temporary file name
    tmp_name = str(uuid.uuid4()) + '.tif'
    
    # clip and resample with gdal
    cmd = 'gdalwarp -te {:.20f} {:.20f} {:.20f} {:.20f} -ts {} {} -t_srs EPSG:4326 -co COMPRESS=LZW -wm 4096 -multi {} {}' \
      .format(xd.longitude.values.min(), xd.latitude.values.min(), xd.longitude.values.max(), xd.latitude.values.max(),
              len(xd.longitude), len(xd.latitude),
              tif_path, tmp_name)
    os.system(cmd)
    
    # load
    with rasterio.open(tmp_name, driver='GTiff') as dst:
        data_np_arr = dst.read()
        dst.close()
    os.remove(tmp_name)
        
    da = xr.DataArray(data_np_arr, dims=['time', 'latitude', 'longitude'])
    da = da.assign_coords(time = range(data_np_arr.shape[0]),
                                      latitude=xd.latitude,
                                      longitude=xd.longitude)
    
    return da

# LOAD LAND COVER
# # V1 using ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992_2015-v2.0.7.tif
# lc = clip_resample_load(NDVI['mean'], lc_path)

# # geotiff is composed of one band per year for the period 1992-2015
# # Assign years to time (index so far) and keep only the appropriate period
# lc.coords['time'] = range(1992, 2015 + 1)
# lc = lc[lc.coords['time'].isin(range(start_year, end_year + 1))]

# # reclassify lc (first year) to ipcc class
# # some idx categories are missning (0 values and 200 cat added and reclassified as 0)
# idx = np.asarray([10, 11, 12, 20, 30, 40, 50, 60, 61, 62, 70, 71, 72, 80, 81, 82, 90, 100, 160, 170, 110, 130, 180, 190, 120, 121, 122, 140, 150, 151, 152, 153, 200, 201, 202, 210, 220])
# val = np.asarray([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 0])
# lookup_array = np.zeros(idx.max() + 1)
# lookup_array[idx] = val
# lookup_array = lookup_array[lc.isel(time=0).values.astype(np.uint8)].astype(np.uint8)
# lc_t0 = xr.DataArray(lookup_array, dims=['latitude', 'longitude'])
# lc_t0 = lc_t0.assign_coords(latitude=ndvi_mean.latitude,
#                             longitude=ndvi_mean.longitude)
# v2 using CH_lc_TE.tif
lc_t0 = clip_resample_load(NDVI['mean'], lc_path)
lc_t0 = lc_t0.where(lc_t0 != -32768, 0).isel(time=0)

# LOAD SOIL TAXONOMY
# # V1 using TAXNWRB_250m_ll.tif
# soil_tax = clip_resample_load(NDVI['mean'], soil_path)
# soil_tax = soil_tax.where(soil_tax.values != 255)

# v2 using CH_soil_tax_TE.tif
soil_tax = clip_resample_load(NDVI['mean'], soil_path)
soil_tax = soil_tax.where(soil_tax != -32768, 0).isel(time=0)


# define unit of analysis as the intersect of soil_tax_usda and land cover
# # V1
# units = soil_tax.isel(time=0).astype(np.uint64).values * 100 + lc_t0
# V2
units = (soil_tax.astype(np.uint64).values * 100 + lc_t0).astype(np.uint16)
units.name = 'units'
# V1
productivity = productivity.merge(units.to_dataset(name = 'units')) # V1
# V2
# da2raster(units, '%s/perf_units.tif' % (work_path),
da2raster(units.where(units != 0), '%s/perf_units.tif' % (work_path),
          nodata_val = 9999,
          md_rast = md_traj,
          md_band = {'long_name':'Soil/LC units', 'units':'id'},
          opt = ['COMPRESS=DEFLATE'])

# compute 90th statistique per unit
ndvi_id = ndvi_avg.to_dataset(name = 'ndvi')
ndvi_id = ndvi_id.merge(units.to_dataset(name = 'units'))
perc90 = ndvi_id.groupby('units').reduce(np.quantile, q=0.9)

# remap perc90 on units
idx = perc90.units.values
val = (perc90.ndvi.values * 100000).astype(np.uint64)
lookup_array = np.zeros(int(idx.max() + 1))
lookup_array[idx] = val
# # V1
# lookup_array = lookup_array[units]
# V2
lookup_array = lookup_array[units.values]
raster_perc = xr.DataArray(lookup_array / 100000, dims=['latitude', 'longitude'])
raster_perc = raster_perc.assign_coords(latitude=ndvi_mean.latitude,
                                        longitude=ndvi_mean.longitude)
ndvi_id = ndvi_id.merge(raster_perc.to_dataset(name = 'perc90'))

# compute degradation
ndvi_id['obs_ratio'] = ndvi_id.ndvi / ndvi_id.perc90
productivity = productivity.merge(ndvi_id['obs_ratio'].to_dataset(name = 'obs_ratio'))
# # V1
# da2raster(ndvi_id['obs_ratio'], '%s/perf_ratio.tif' % (work_path),
# V2
da2raster(ndvi_id['obs_ratio'].where(units != 0), '%s/perf_ratio.tif' % (work_path),
          nodata_val = 9999,
          md_rast = md_traj,
          md_band = {'long_name':'NDVI average / perc 90 ratio', 'units':'percentage'},
          opt = ['COMPRESS=DEFLATE'])

# compute degradation using obs_ratio = 0.5 as a threshold
deg = np.zeros((ndvi_mean.shape[1], ndvi_mean.shape[2]))
deg = np.where(ndvi_id.obs_ratio <= 0.5, -1, 0)
deg = xr.DataArray(deg, dims=['latitude', 'longitude']).astype(np.int16) # -1 becomes nodata when using np.int8
deg = deg.assign_coords(latitude=ndvi_id.latitude, longitude=ndvi_id.longitude)
deg.name = 'degradation'
productivity = productivity.merge(deg.to_dataset(name = 'deg'))
# # V1
# da2raster(deg, '%s/perf_deg.tif' % (work_path),
# V2
da2raster(deg.where(units != 0), '%s/perf_deg.tif' % (work_path),
          nodata_val = 9999,
          md_rast = md_traj,
          md_band = {'long_name':'degradation', 'units':'category'},
          opt = ['COMPRESS=DEFLATE'])
del units
del ndvi_id
del deg

print(productivity)

### Compute state <a name="state"></a>[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

In [ ]:
%%time

# performance metadata for geotiff export
md_traj = {'description':'productivity-state indicators using Landsat 5,7 and 8 using the period 2001-2012 as baseline and the period 2013-2015 as a target'}

# Define periods and create subsets of mean annual NDVI values for each period
bl_ndvi_range = ndvi_mean.sel(time=slice(start_year,bl_end_year)) # bl_ (Baseline period)
tg_ndvi_range = ndvi_mean.sel(time=slice(tg_start_year,end_year)) # tg_ (Target period)

# Calculate min and max for baseline period:
bl_ndvi_min = bl_ndvi_range.min(dim=['time'])
bl_ndvi_max = bl_ndvi_range.max(dim=['time'])

# Extend the min and max by 5 % for the baseline period
bl_extended_perc_5 = bl_ndvi_min - ((bl_ndvi_max - bl_ndvi_min) * 0.05)
bl_extended_perc_105 = bl_ndvi_max + ((bl_ndvi_max - bl_ndvi_min) * 0.05)

# Generate extreme values (+/- 5% extended values) with the baseline mean NDVI subset in new
# datAarray with new "ind" coordinate
bl_extended = [None]*(len(bl_ndvi_range['time']) + 2)
years = range(start_year, bl_end_year + 1)
for year_ind, year in enumerate(years):
    bl_extended[year_ind] = bl_ndvi_range.sel(time=years[year_ind])
    bl_extended[year_ind] = bl_extended[year_ind].assign_coords(ind=year_ind).drop('time')
p5 = year_ind+1
p105 = year_ind+2
bl_extended[p5] = bl_extended_perc_5.assign_coords(ind=p5)
bl_extended[p105] = bl_extended_perc_105.assign_coords(ind=p105)
bl_extended = xr.concat(bl_extended, dim ='ind')

# Define the percentiles of the extended range of annual mean ndvi values in the baseline period
# using np.quantile on "cleaned" data (dropna) instead of xr.quantile or np.naquantile (both ~300x faster)
# bl_quant =  bl_extended.quantile(quantiles, dim='ind')
# bl_quant = np.nanquantile(bl_extended, quantiles, axis = 0)
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
bl_quant = np.quantile(bl_extended.dropna('ind', how='all'), quantiles, axis = 0)
bl_quant = xr.DataArray(bl_quant, dims=['quantile', 'latitude', 'longitude'])
bl_quant = bl_quant.assign_coords(quantile = quantiles,
                                  latitude=bl_extended.latitude,
                                  longitude=bl_extended.longitude)

# Calculate mean NDVI for baseline and target periods
bl_ndvi_mean = bl_ndvi_range.mean(dim='time').rename('NDVI_mean')
tg_ndvi_mean = tg_ndvi_range.mean(dim='time').rename('NDVI_mean')

# Reclassify the baseline mean ndvis based on the quantiles
bl_classes = np.full((bl_ndvi_mean.shape[0], bl_ndvi_mean.shape[1]), 9999)
bl_classes=np.where(bl_ndvi_mean <= bl_quant.sel(quantile=0.1), 1, bl_classes)
bl_classes=np.where(bl_ndvi_mean > bl_quant.sel(quantile=0.1), 2, bl_classes)
bl_classes=np.where(bl_ndvi_mean > bl_quant.sel(quantile=0.2), 3, bl_classes)
bl_classes=np.where(bl_ndvi_mean > bl_quant.sel(quantile=0.3), 4, bl_classes)
bl_classes=np.where(bl_ndvi_mean > bl_quant.sel(quantile=0.4), 5, bl_classes)
bl_classes=np.where(bl_ndvi_mean > bl_quant.sel(quantile=0.5), 6, bl_classes)
bl_classes=np.where(bl_ndvi_mean > bl_quant.sel(quantile=0.6), 7, bl_classes)
bl_classes=np.where(bl_ndvi_mean > bl_quant.sel(quantile=0.7), 8, bl_classes)
bl_classes=np.where(bl_ndvi_mean > bl_quant.sel(quantile=0.8), 9, bl_classes)
bl_classes=np.where(bl_ndvi_mean > bl_quant.sel(quantile=0.9), 10, bl_classes)

# Reclassify the target mean ndvis based on the quantiles
tg_classes = np.full((tg_ndvi_mean.shape[0], tg_ndvi_mean.shape[1]), 9999)
tg_classes=np.where(tg_ndvi_mean <= bl_quant.sel(quantile=0.1), 1, tg_classes)
tg_classes=np.where(tg_ndvi_mean > bl_quant.sel(quantile=0.1), 2, tg_classes)
tg_classes=np.where(tg_ndvi_mean > bl_quant.sel(quantile=0.2), 3, tg_classes)
tg_classes=np.where(tg_ndvi_mean > bl_quant.sel(quantile=0.3), 4, tg_classes)
tg_classes=np.where(tg_ndvi_mean > bl_quant.sel(quantile=0.4), 5, tg_classes)
tg_classes=np.where(tg_ndvi_mean > bl_quant.sel(quantile=0.5), 6, tg_classes)
tg_classes=np.where(tg_ndvi_mean > bl_quant.sel(quantile=0.6), 7, tg_classes)
tg_classes=np.where(tg_ndvi_mean > bl_quant.sel(quantile=0.7), 8, tg_classes)
tg_classes=np.where(tg_ndvi_mean > bl_quant.sel(quantile=0.8), 9, tg_classes)
tg_classes=np.where(tg_ndvi_mean > bl_quant.sel(quantile=0.9), 10, tg_classes)

# Convert the new arrays to proper DataArrays (cause they're missing coordinates after the calculation)
bl_classes = xr.DataArray(bl_classes, dims=['latitude', 'longitude']).astype(np.int16)
bl_classes = bl_classes.assign_coords(latitude=bl_ndvi_mean.latitude, longitude=bl_ndvi_mean.longitude)
bl_classes.name = 'bl_classes'
productivity = productivity.merge(bl_classes.to_dataset(name = 'bl_classes'))
da2raster(bl_classes, '%s/stat_bl_classes.tif' % (work_path), nodata_val = 9999,
          md_rast = md_traj,
          md_band = {'long_name':'baseline NDVI percentile', 'units':'class'},
          opt = ['COMPRESS=DEFLATE'])
tg_classes = xr.DataArray(tg_classes, dims=['latitude', 'longitude']).astype(np.int16)
tg_classes = tg_classes.assign_coords(latitude=tg_ndvi_mean.latitude, longitude=tg_ndvi_mean.longitude)
tg_classes.name = 'tg_classes'
productivity = productivity.merge(tg_classes.to_dataset(name = 'tg_classes'))
da2raster(tg_classes, '%s/stat_tg_classes.tif' % (work_path), nodata_val = 9999,
          md_rast = md_traj,
          md_band = {'long_name':'target NDVI percentile', 'units':'class'},
          opt = ['COMPRESS=DEFLATE'])

# Compare the difference between the baseline and the target mean NDVI values based on the quantile classes
# with the exception of subtle changes of mean NDVI value (<0.01) which gets a "0"
classes_chg = tg_classes - bl_classes
classes_chg = np.where(abs(bl_ndvi_mean - tg_ndvi_mean) <= 0.01, 0, classes_chg)

# Convert the new arrays to proper DataArrays (cause they're missing coordinates after the calculation)
classes_chg = xr.DataArray(classes_chg, dims=['latitude', 'longitude']).astype(np.int16)
classes_chg = classes_chg.assign_coords(latitude=bl_classes.latitude, longitude=bl_classes.longitude)
classes_chg.name = 'classes_chg'
productivity = productivity.merge(classes_chg.to_dataset(name = 'classes_chg'))
da2raster(classes_chg, '%s/stat_classes_chg.tif' % (work_path), nodata_val = 9999,
          md_rast = md_traj,
          md_band = {'long_name':'NDVI percentile difference', 'units':'class'},
          opt = ['COMPRESS=DEFLATE'])

# Normalize the classes_chg dataarray in order to consider that <=-2 is degradation
classes_chg_norm = np.full((classes_chg.shape[0], classes_chg.shape[1]), 9999)
classes_chg_norm = np.where(classes_chg < 2, 0, classes_chg_norm)
classes_chg_norm = np.where(classes_chg <= -2, -1, classes_chg_norm)
classes_chg_norm = np.where(classes_chg >=2, 1, classes_chg_norm)

# Convert the new arrays to proper DataArrays (cause they're missing coordinates after the calculation)
classes_chg_norm = xr.DataArray(classes_chg_norm, dims=['latitude', 'longitude']).astype(np.int16)
classes_chg_norm = classes_chg_norm.assign_coords(latitude=classes_chg.latitude, longitude=classes_chg.longitude)
classes_chg_norm.name = 'classes_chg_norm'
productivity = productivity.merge(classes_chg_norm.to_dataset(name = 'classes_chg_norm'))
da2raster(classes_chg_norm, '%s/stat_classes_chg_norm.tif' % (work_path), nodata_val = 9999,
          md_rast = md_traj,
          md_band = {'long_name':'State', 'units':'category'},
          opt = ['COMPRESS=DEFLATE'])
del bl_classes
del tg_classes
del classes_chg
del classes_chg_norm

print(productivity)

### Combine productivity indicators <a name="combine"></a>[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

In [ ]:
%%time

# Final calculation of productivity
# 3 categories(improving = 1, degrading = -3, stable = 0)
# plus 2 optional categories (stable but stressed = -1, early signs of decline = -2)

combination = np.full((NDVI.dims['latitude'], NDVI.dims['longitude']), 9999)

# Addressing....
# lines 1 to 5
combination = np.where(np.logical_and(productivity.signif.values > 1,
                                      np.logical_not(productivity.signif.values == 9999)), 1, combination)
# lines 13 to 18
combination = np.where((productivity.signif.values < -1), -3, combination)
# lines 7 to 9
combination = np.where(np.logical_and(productivity.signif.values < 2,
                                      productivity.signif.values > -2), 0, combination)
# lines 6, 12 (and 18 !)
combination = np.where(np.logical_and(productivity.deg.values < 0,
                                      productivity.classes_chg.values < -1), -3, combination)

# Separation in 2 versions of productivity (3 classes or 5 classes)
#Prod_3
# lines 10 and 11
combination_3 = np.where(np.logical_and(np.logical_and(combination == 0,
                                                       np.logical_or(productivity.deg.values < 0,
                                                                     productivity.classes_chg.values < -1)),
                                        np.logical_not(productivity.classes_chg.values > 1)), -3, combination)

#Prod_5
# line 6 (different than prod_3)
combination_5 = np.where(np.logical_and(np.logical_and(productivity.signif.values > 1,
                                                       np.logical_not(productivity.signif.values == 9999)),
                                        np.logical_and(productivity.classes_chg.values < -1,
                                                       productivity.deg.values < 0)), 0, combination)
# line 10 (needed to add a fake clause for the second logical_and)
combination_5 = np.where(np.logical_and(np.logical_and(combination == 0,
                                                       productivity.deg.values == -1),
                                        np.logical_and(productivity.classes_chg.values < 2,
                                                       productivity.classes_chg.values > -2)), -1, combination_5)
# line 11 
combination_5 = np.where(np.logical_and(combination == 0,
                                        np.logical_and(productivity.classes_chg.values < -1,
                                                       productivity.deg.values == 0)), -2, combination_5)

# Convert propper DataArray
md_traj = {'description':'3 classes productivity using Landsat 5,7 and 8 using the period 2001-2012 as baseline and the period 2013-2015 as a target'}

combination_3 = xr.DataArray(combination_3, dims=['latitude', 'longitude']).astype(np.int16)
combination_3 = combination_3.assign_coords(latitude=NDVI.latitude, longitude=NDVI.longitude)
combination_3.name = 'combination_3'
productivity = productivity.merge(combination_3.to_dataset(name = 'combination_3'))
da2raster(combination_3, '%s/prod_combination_3.tif' % (work_path), nodata_val = 9999,
          md_rast = md_traj,
          md_band = {'long_name':'combination_3', 'units':'categories'},
          opt = ['COMPRESS=DEFLATE'])
del combination_3
combination_5 = xr.DataArray(combination_5, dims=['latitude', 'longitude']).astype(np.int16)
combination_5 = combination_5.assign_coords(latitude=NDVI.latitude, longitude=NDVI.longitude)
combination_5.name = 'combination_5'
productivity = productivity.merge(combination_5.to_dataset(name = 'combination_5'))
da2raster(combination_5, '%s/prod_combination_5.tif' % (work_path), nodata_val = 9999,
          md_rast = md_traj,
          md_band = {'long_name':'combination_5', 'units':'categories'},
          opt = ['COMPRESS=DEFLATE'])
del combination_5

print(productivity)

In [ ]:
printandlog('ALL DONE\n', log_name)

if user_mail != '':
    os.system("mail -s 'Script BC_TrendsEarth_productivity_v2 completed' %s" % (user_mail))

### Visualize productivity (OPTIONAL)<a name="visualize"></a>[<div style="text-align: right; font-size: 24px"> &#x1F51D; </div>](#top)

**_!!! Full section to be commented in case of large dataset !!!_**